In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from datetime import datetime as dt
import itertools

%matplotlib inline

# Load the Dataset

In [9]:
df = pd.read_csv("../datasets/FIFA 1990-2018 - Cleaned.csv")
squads = pd.read_csv("../datasets/New_data/2018 FIFA World Cup Squads.csv")
fifa18 = pd.read_csv("../datasets/New_data/complete.csv")
distance = pd.read_csv("../datasets/New_data/distance.csv", error_bad_lines=False)
fifa_ranking = pd.read_csv("../datasets/New_data/fifa_ranking.csv")

b'Skipping line 31: expected 4 fields, saw 5\nSkipping line 63: expected 4 fields, saw 5\nSkipping line 85: expected 4 fields, saw 5\nSkipping line 102: expected 4 fields, saw 5\nSkipping line 173: expected 4 fields, saw 5\nSkipping line 174: expected 4 fields, saw 5\nSkipping line 196: expected 4 fields, saw 5\nSkipping line 203: expected 4 fields, saw 5\nSkipping line 204: expected 4 fields, saw 5\nSkipping line 207: expected 4 fields, saw 5\nSkipping line 211: expected 4 fields, saw 5\nSkipping line 213: expected 4 fields, saw 5\n'


# Team Features

In [10]:
df.head()

,Team1,Team2,Rounds,Stadium,City,Time,Day,Month,Year,Score1,Score2,Remarks
0,Russia,Saudi Arabia,Group Phase,Luzhniki Stadium,Moscow,18:00,14,6,2018,5,0,NaN
1,Egypt,Uruguay,Group Phase,Ekaterinburg Arena,Ekaterinburg,17:00,15,6,2018,0,1,NaN
2,Morocco,IR Iran,Group Phase,Saint Petersburg Stadium,St. Petersburg,18:00,15,6,2018,0,1,NaN
3,Portugal,Spain,Group Phase,Fisht Stadium,Sochi,21:00,15,6,2018,3,3,{WinTeamName} win on aggregate after regular t...
4,France,Australia,Group Phase,Kazan Arena,Kazan,13:00,16,6,2018,2,1,NaN


### To do :
- Age  avg per team per position
- Caps avg per team
- Avg goal for all member
- team was until now in the 1/4 round ? True =1 or False =0
    

In [15]:
#Adding goal difference and establishing who is the winner 
winner = []
for i in range (len(df['Team1'])):
    if df['Score1'][i] > df['Score2'][i]:
        winner.append(df['Team1'][i])
    elif df['Score1'][i] < df['Score2'][i]:
        winner.append(df['Team2'][i])
    else:
        winner.append('Draw')
df['winning_team'] = winner

#adding goal difference column
df['goal_difference'] = np.absolute(df['Score1'] - df['Score2'])

df.head()

,Team1,Team2,Rounds,Stadium,City,Time,Day,Month,Year,Score1,Score2,Remarks,winning_team,goal_difference
0,Russia,Saudi Arabia,Group Phase,Luzhniki Stadium,Moscow,18:00,14,6,2018,5,0,NaN,Russia,5
1,Egypt,Uruguay,Group Phase,Ekaterinburg Arena,Ekaterinburg,17:00,15,6,2018,0,1,NaN,Uruguay,1
2,Morocco,IR Iran,Group Phase,Saint Petersburg Stadium,St. Petersburg,18:00,15,6,2018,0,1,NaN,IR Iran,1
3,Portugal,Spain,Group Phase,Fisht Stadium,Sochi,21:00,15,6,2018,3,3,{WinTeamName} win on aggregate after regular t...,Draw,0
4,France,Australia,Group Phase,Kazan Arena,Kazan,13:00,16,6,2018,2,1,NaN,France,1


In [64]:
#AVG Age of each team in each position
g2=squads.groupby(["Team","Position"])['Age'].sum()/squads.groupby(["Team","Position"])['Age'].size()
g2.to_csv("./team_avg_age.csv",sep="\t")

In [93]:
#AVG Caps of each team per position
squads.groupby(["Team","Position"])['Caps'].sum()/squads.groupby(["Team","Position"])['Caps'].size()


Team         Position
Argentina    DF          34.444444
             FW          73.000000
             GK           3.000000
             MF          32.714286
Australia    DF          22.428571
             FW          33.250000
             GK          16.333333
             MF          39.400000
Belgium      DF          48.857143
             FW          46.200000
             GK          25.666667
             MF          46.750000
Brazil       DF          31.125000
             FW          30.200000
             GK           9.000000
             MF          34.714286
Colombia     DF          21.857143
             FW          29.600000
             GK          30.666667
             MF          40.750000
Costa Rica   DF          40.777778
             FW          60.666667
             GK          42.333333
             MF          60.875000
Croatia      DF          37.750000
             FW          41.500000
             GK          16.000000
             MF          51.83333

# Player Features

In [12]:
fifa18 = fifa18.replace({"Korea Republic":"South Korea"})
wc_player_stats_18 = fifa18[fifa18["name"].isin(squads.Player) | fifa18["full_name"].isin(squads.Player)]
wc_fifa18_stats = fifa18[fifa18.nationality.isin(squads.Team)]

In [15]:
wc_fifa18_stats.replace([False,"False",True,"True"],[0,0,1,1])
unused_var=[]
for i in fifa18.columns:
    if "prefers" in i:
        unused_var.append(i)
wc_fifa18_stats = wc_fifa18_stats.drop(["club_logo","ID","real_face","birth_date","flag","photo"],axis=1)
wc_fifa18_stats = wc_fifa18_stats.drop(unused_var,axis=1)
grouped_wc_fifa18_stats = wc_fifa18_stats.groupby(['nationality']).apply(lambda x: (x.sort_values('overall',ascending=False)).head(30).mean()).sort_values('potential',ascending=True)
grouped_wc_fifa18_stats.head()

,special,age,height_cm,weight_kg,eur_value,eur_wage,eur_release_clause,overall,potential,pac,...,playmaker_speciality,engine_speciality,distance_shooter_speciality,crosser_speciality,free_kick_specialist_speciality,tackling_speciality,tactician_speciality,acrobat_speciality,strength_speciality,clinical_finisher_speciality
nationality,,,,,,,,,,,,,,,,,,,,,
Panama,1617.375000,25.812500,181.750000,76.500000,8.378125e+05,3312.500000,1.344812e+06,65.812500,69.437500,66.062500,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.062500,0.0
Tunisia,1677.966667,27.266667,182.700000,76.666667,2.261167e+06,11700.000000,2.739000e+06,68.966667,71.533333,66.200000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.033333,0.0
Costa Rica,1695.966667,26.433333,181.000000,75.400000,3.196167e+06,18033.333333,5.612536e+06,69.900000,72.000000,73.400000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.066667,0.0
Iceland,1771.000000,27.233333,184.066667,78.200000,2.607500e+06,16700.000000,4.797357e+06,70.033333,72.166667,67.766667,...,0.0,0.1,0.0,0.033333,0.033333,0.0,0.0,0.000000,0.000000,0.0
Peru,1651.965517,25.517241,178.931034,73.896552,2.122069e+06,6620.689655,5.940867e+06,69.137931,72.689655,69.551724,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.034483,0.000000,0.0


In [18]:
grouped_wc_fifa18_stats.shape

(32, 143)

In [30]:
distance.head()

,Country,Russia,Distance_km,Mileage
0,Andorra,Russia,"6,597","4,099"
1,United Arab Emirates,Russia,"5,731","3,561"
2,Afghanistan,Russia,"4,056","2,520"
3,Antigua and Barbuda,Russia,"11,202","6,961"
4,Anguilla,Russia,"11,088","6,890"
